# Pipeline training

This notebook is used for triggering the training pipeline on the basis of the steps described in the notebook mlops_dnn.ipynb

In [ ]:
import boto3
import json
import logging
import os
from sagemaker import get_execution_role
import sys

In [ ]:
sys.path.insert(0, os.path.abspath('./../mlpipelines/pipelines'))

In [ ]:
from training.pipeline import get_pipeline

***

In [ ]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

***

## Global parameters

In [ ]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
role = get_execution_role()
kms_alias = "ml-kms-dev"
lambdas = {
    "create-compilation-job": "",
    "check-compilation-job": "",
}
processing_ecr_image_name = ""
processing_ecr_image_tag = "latest"
sagemaker_framework_version = "2.4"
sagemaker_python_version = "py37"
s3_bucket_name = "isengard-bpistone-mlops-iot-dev"
s3_processing_input_files_path = "input/data"
s3_processing_output_files_path = "output/data"
s3_training_artifact_path = "artifact/training"
s3_training_artifact_name = "sourcedir_dnn.tar.gz"
s3_training_input_files_path = "output/data"
s3_training_output_files_path = "output/model"

***

## Training pipeline

### Get pipeline definition

In [ ]:
pipeline = get_pipeline(
    region,
    account_id,
    kms_alias,
    lambdas,
    sagemaker_framework_version,
    sagemaker_python_version,
    s3_bucket_name,
    s3_training_artifact_path,
    s3_training_artifact_name,
    s3_training_input_files_path,
    s3_training_output_files_path
)

### Create or update SageMaker pipeline

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
json.loads(pipeline.definition())

### Pipeline parameters

In [ ]:
compilation_input_shape = "[1, 1, 1, 1]"
input_file_name = "processed.csv"
instance_type = "ml.m5.large"
instance_count = 1
processing_input_file_name = "bottle.csv"
processing_instance_count = 1
processing_instance_type = "ml.t3.large"
model_package_group_name = "mlops-iot-regressor"

### Start training pipeline 

In [ ]:
execution = pipeline.start(
    parameters=dict(
        compilation_input_shape=compilation_input_shape,
        input_file_name=input_file_name,
        model_package_group_name=model_package_group_name,
        training_instance_count=instance_count,
        training_instance_type=instance_type
    )
)

In [ ]:
execution.describe()

In [ ]:
execution.list_steps()